환경설정

In [6]:
!pip install PyMuPDF

   ---------------------------------------- 0.0/16.0 MB ? eta -:--:--
   -- ------------------------------------- 1.0/16.0 MB 12.5 MB/s eta 0:00:02
   ---- ----------------------------------- 1.8/16.0 MB 4.0 MB/s eta 0:00:04
   -------------- ------------------------- 5.8/16.0 MB 9.3 MB/s eta 0:00:02
   ------------------ --------------------- 7.3/16.0 MB 10.1 MB/s eta 0:00:01
   ------------------------ --------------- 9.7/16.0 MB 9.3 MB/s eta 0:00:01
   ----------------------------- ---------- 11.8/16.0 MB 9.5 MB/s eta 0:00:01
   -------------------------------------- - 15.5/16.0 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 16.0/16.0 MB 9.7 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

문서 로드

In [7]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/snow-white.pdf")

docs = loader.load()

print(f"문서의 페이지 수 : {len(docs)}")

문서의 페이지 수 : 6


In [8]:
print(docs[0].page_content)

백설공주
옛날어느왕국에공주님이태어났어요.
“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백
설공주라고불러야겠다.”
왕과왕비는갓태어난딸을보며기뻐했어요.
하지만기쁨도잠시, 왕비는곧세상을떠나고말았어
요.



In [9]:
print(docs[0].__dict__)

{'id': None, 'metadata': {'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, 'page_content': '백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.\n하지만기쁨도잠시, 왕비는곧세상을떠나고말았어\n요.\n', 'type': 'Document'}


문서 분할

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 21


임베딩

In [18]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

DB생성

In [15]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   --------- ------------------------------ 3.7/14.9 MB 18.2 MB/s eta 0:00:01
   ---------------------- ----------------- 8.4/14.9 MB 22.6 MB/s eta 0:00:01
   ---------------------------------------  14.7/14.9 MB 28.0 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 24.6 MB/s eta 0:00:00


In [25]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=split_documents, embedding = embeddings)

In [26]:
for doc in vectorstore.similarity_search("난쟁이"):
    print(doc.page_content)

저녁이되자, 일곱난쟁이가돌아왔어요.
난쟁이들은쓰러진백설공주를보고엉엉울었어요.
백설공주는깊은잠에빠진것처럼보였지요.
“백설공주님, 못된왕비의꾐에넘어갔군요.”
간사과랍니다. 잠깐문을열어보세요.”
백설공주는고개를저었어요.
“난쟁이들이문을열어주지말라고했어요.”
백설공주가거절하자, 왕비는창문틈새로사과를쑥내밀었어
요.
왕자는깨어난백설공주를보고기뻐했어요.
“공주님, 나는이웃나라왕자입니다.”
“왕자님이나를다시살려주셨군요.”
“나와결혼해주시겠어요?”
“네, 좋아요!”
밤이되자오두막주인인일곱난쟁이가돌아왔어요.
난쟁이들은집안이어질러진것을보고깜짝놀랐지요.
일곱째난쟁이가큰소리로외쳤어요.
“누가내침대에서자고있어!”


In [27]:
retriver = vectorstore.as_retriever()

In [28]:
retriver.invoke("백설공주와 일곱난쟁이는 어디서 만났어?")

[Document(metadata={'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, page_content='백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.'),
 Document(metadata={'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 4, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, page_content='저녁이되자, 일곱난쟁이가돌아왔어요.\n난쟁이들은쓰러진백설공주를보고엉엉울었어요.\n백설공주는깊은잠에빠진것처럼보였지요.\n“백설공주님, 못된왕비의꾐에넘어갔군요.

프롬프트 생성

당신은 질문 답변 작업을 위한 어시스턴트입니다
주어진 문맥을 사용하여 질문에 답변하세요
유치원 선생님이 아이에게 말하는 것처럼 매우 친절하고 부드러운 어조를 사용하세요
따뜻하고 친근한 방식으로 말하세요
답을 모르는 경우에는 모른다고 말하세요
한국어로 말하세요

In [29]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
Use a very kind and gentle tone like a kindergarten teacher talking to a child.
Speak in a warm and friendly way.
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:
"""
)

모델생성

In [30]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

chain 생성

In [39]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": retriver, "question": RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()

)

In [40]:
question = "백설공주와 왕비 중 누가 더 아름더워?"
response = chain.invoke(question)

print(response)

친구야, 백설공주가 왕비보다 더 아름답다고 요술거울이 말했단다. 그래서 왕비님이 조금 화가 나셨지요. 하지만 우리 모두는 각자 다르게 아름답다는 걸 기억해야 해요. 😊


10월호 산업동향 관련 실습

In [45]:
from langchain_community.document_loaders import PyMuPDFLoader

PracticeLoader = PyMuPDFLoader("data/SPRi AI Brief_10월호_산업동향_F.pdf")

PracticeDocs = PracticeLoader.load()

print(f"문서의 페이지 수 : {len(PracticeDocs)}")

문서의 페이지 수 : 25


In [46]:
print(PracticeDocs[0].page_content)

2024년 10월호



In [47]:
print(PracticeDocs[0].__dict__)

{'id': None, 'metadata': {'source': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'file_path': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 0, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241007090546+09'00'", 'modDate': "D:20241007090546+09'00'", 'trapped': ''}, 'page_content': '2024년 10월호\n', 'type': 'Document'}


In [48]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

practice_split_documents = text_splitter.split_documents(PracticeDocs)

print(f"분할된 청크의 수 : {len(practice_split_documents)}")

분할된 청크의 수 : 499


In [50]:
from langchain_openai import OpenAIEmbeddings

practiceEmbeddings = OpenAIEmbeddings()

In [51]:
from langchain_community.vectorstores import FAISS

practiceVectorstore = FAISS.from_documents(documents=practice_split_documents, embedding = practiceEmbeddings)

In [52]:
for PracticeDoc in practiceVectorstore.similarity_search("거버넌스"):
    print(PracticeDoc.page_content)

∙인물 이미지 생성 기능은 제미나이 어드밴스드, 비즈니스, 엔터프라이즈 이용자를 대상으로 영어로 
우선 제공되며 향후 지원 언어와 이용자 범위를 확대할 계획
실현하기 위한 권고안을 담은 ‘인류를 위한 AI 거버넌스’ 최종 보고서를 발간
방식 대신 다른 방식을 시도하는 방법 등을 학습해 추론 능력을 비약적으로 향상
애니메이션 등 다양한 스타일의 이미지를 생성하며, 이마젠 2보다 이미지 생성 기능이 대폭 향상


In [55]:
practiceRetriever = practiceVectorstore.as_retriever()

In [56]:
practiceRetriever.invoke("GPU 클러스터는 어디에 집중되어 있어?")

[Document(metadata={'source': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'file_path': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 14, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241007090546+09'00'", 'modDate': "D:20241007090546+09'00'", 'trapped': ''}, page_content='n 옥스퍼드⼤ 연구진에 따르면 AI 개발에 필수적인 GPU 클러스터는 미국과 중국을 중심으로 \n전 세계 30개국에 집중되어 있으며, 대부분 지역에는 GPU 클러스터가 부재'),
 Document(metadata={'source': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'file_path': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 14, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241007090546+09'00'", 'modDate': "D:20241007090546+09'00'", 'trapped': ''}, page_content='∙중국은 GPU 클러스터를 보유한 지역 수에서 미국을 앞서나, 대중국 수출이 제한된 엔비디아\n(NVIDIA)의 H100과 같은 

In [63]:
from langchain_core.prompts import PromptTemplate
practicePrompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
Use a very kind and gentle tone like a kindergarten teacher talking to a child.
Speak in a warm and friendly way.
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{practiceContext}

#Question:
{practiceQuestion}

#Answer:
""")

In [64]:
from langchain_openai import ChatOpenAI
practiceLlm = ChatOpenAI(model="gpt-4o", temperature=0)

In [65]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"practiceContext": retriver, "practiceQuestion": RunnablePassthrough()}
    |practicePrompt
    |practiceLlm
    |StrOutputParser()

)

In [68]:
practiceQuestion = "로니콜먼의 3대중량은 어떻게 돼?"
practiceResponse = chain.invoke(practiceQuestion)

print(practiceResponse)

미안하지만, 로니 콜먼의 3대 중량에 대한 정보는 여기에서 찾을 수 없어요. 다른 질문이 있으면 언제든지 물어봐 주세요!


In [69]:
practiceQuestion = "코히어 연구는 어느나라에서 성공했어?"
practiceResponse = chain.invoke(practiceQuestion)

print(practiceResponse)

코히어 연구는 캐나다에서 성공했어요. 😊
